In [1]:
import os
import pandas as pd
from collections import defaultdict
from functools import partial
from multiprocessing import cpu_count
from pathlib import Path
from textwrap import dedent
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import torch
from torch import nn
from torch import optim
from torch.nn import functional as F
from torch.optim.lr_scheduler import _LRScheduler
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data.dataset import Dataset
import torch as tf
from collections import Counter

In [2]:
def compute_label(input_front, input_back, convert_dict):
    if input_front['write'] > 0.9:
        out_label = convert_dict['write']
    elif input_front['talk'] > 0.99:
        out_label = convert_dict['talk']
    elif input_front['read'] > 0.5:
        out_label = convert_dict['read']
    elif input_front['drink'] > 0.9:
        out_label = convert_dict['drink']
    elif input_front['eat'] > 0.4:
        out_label = convert_dict['eat']
    elif input_back['computer'] > 0.6:
        out_label = convert_dict['computer']
    else:
        out_label = 6
    return out_label

In [3]:
def load_sensor_data_without_h(fname):
    sensor_txt = np.genfromtxt(fname, delimiter=',', dtype=None, encoding=None)
    # a 2-4 w(augular velocity) 6-8 Angle 10-12 h 14-16 q(quaternion) 18-21
    # a 0-2 w 3-5 Angle 6-8 q 9 10 11 12
    row_len = 3*3 + 4
    data_length = len(sensor_txt)
    sensor_array = np.zeros((data_length, row_len))
    for row_i, sensor_row in enumerate(sensor_txt):
        # a 2-4
        sensor_array[row_i, 0] = sensor_row[2]
        sensor_array[row_i, 1] = sensor_row[3]
        sensor_array[row_i, 2] = sensor_row[4]
        # w 6-8
        sensor_array[row_i, 3] = sensor_row[6]
        sensor_array[row_i, 4] = sensor_row[7]
        sensor_array[row_i, 5] = sensor_row[8]
        # Angle 10-12
        sensor_array[row_i, 6] = sensor_row[10]
        sensor_array[row_i, 7] = sensor_row[11]
        sensor_array[row_i, 8] = sensor_row[12]
        # q 18-21
        sensor_array[row_i, 9] = sensor_row[18]
        sensor_array[row_i, 10] = sensor_row[19]
        sensor_array[row_i, 11] = sensor_row[20]
        sensor_array[row_i, 12] = sensor_row[21]
    return sensor_array
def sample_sensor_data(input_data, window_sz = 128, sample_sz = 128):
    sensor_length = input_data.shape[0]
#     print('The shape of sensor input data', input_data.shape)
    feature_sz = input_data.shape[1]
    data_sz = 0
#     print('the length of sensor', sensor_length)
    for i in range(0, sensor_length-window_sz-sample_sz, sample_sz):
        data_sz = data_sz + 1
    all_sensor_data = np.zeros((data_sz, feature_sz, window_sz))
    cnt = 0
    for i in range(0, sensor_length-window_sz-sample_sz, sample_sz):
        sample = input_data[i:i + window_sz, :]
        sample = np.transpose(sample)
        all_sensor_data[cnt, :, :] = sample
        cnt = cnt + 1
#     print('the shape of sensor dataset', all_sensor_data.shape)
    return all_sensor_data 

In [4]:
class _SepConv1d(nn.Module):
    """A simple separable convolution implementation.

    The separable convlution is a method to reduce number of the parameters
    in the deep learning network for slight decrease in predictions quality.
    """

    def __init__(self, ni, no, kernel, stride, pad):
        super().__init__()
        self.depthwise = nn.Conv1d(ni, ni, kernel, stride, padding=pad, groups=ni)
        self.pointwise = nn.Conv1d(ni, no, kernel_size=1)

    def forward(self, x):
        return self.pointwise(self.depthwise(x))


class SepConv1d(nn.Module):
    """Implementes a 1-d convolution with 'batteries included'.

    The module adds (optionally) activation function and dropout layers right after
    a separable convolution layer.
    """

    def __init__(self, ni, no, kernel, stride, pad, drop=None,
                 activ=lambda: nn.ReLU(inplace=True)):

        super().__init__()
        assert drop is None or (0.0 < drop < 1.0)
        layers = [_SepConv1d(ni, no, kernel, stride, pad)]
        if activ:
            layers.append(activ())
        if drop is not None:
            layers.append(nn.Dropout(drop))
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)


class Flatten(nn.Module):
    """Converts N-dimensional tensor into 'flat' one."""

    def __init__(self, keep_batch_dim=True):
        super().__init__()
        self.keep_batch_dim = keep_batch_dim

    def forward(self, x):
        if self.keep_batch_dim:
            return x.view(x.size(0), -1)
        return x.view(-1)


class Classifier_dh(nn.Module):
    def __init__(self, raw_ni, raw_sz, no, drop=0.05):
        super().__init__()
        self.conv1 = SepConv1d(raw_ni, 32, 8, 2, 3, drop=drop)
        self.conv2 = SepConv1d(32, 64, 8, 4, 2, drop=drop)
        self.conv3 = SepConv1d(64, 128, 8, 4, 2, drop=drop)
        self.conv4 = SepConv1d(128, 256, 8, 4, 2)
        self.flatten = Flatten()
        self.fc1 = nn.Sequential(nn.Dropout(drop), nn.Linear(raw_sz*2, 64), nn.ReLU(inplace=True))
        self.fc2 = nn.Sequential(nn.Dropout(drop), nn.Linear(64, 64), nn.ReLU(inplace=True))
        self.out = nn.Sequential(
            nn.Linear(64, 64), nn.ReLU(inplace=True), nn.Linear(64, no))

    def forward(self, t_raw):
#         print('input size', t_raw.size())
        x = self.conv1(t_raw)
#         print('conv1', x.size())
        x = self.conv2(x)
#         print('conv2', x.size())
        x = self.conv3(x)
#         print('conv3', x.size())
        x = self.conv4(x)
#         print('conv4', x.size())
        x = self.flatten(x)
#         print('flatten', x.size())
        x = self.fc1(x)
#         print('fc1', x.size())
        x = self.fc2(x)
#         print('fc2', x.size())
        x = self.out(x)
        return x

In [5]:
# Program to find most frequent  
# element in a list 
from collections import Counter
def remove_values_from_list(the_list, val):
    return [value for value in the_list if value != val]
def most_frequent(List):
    List = remove_values_from_list(the_list=List, val=6)
    if List == []:
        return 6
    else:
        occurence_count = Counter(List)
        return occurence_count.most_common(1)[0][0]

In [6]:
#actor 1 male:  lengths of video 1-7 are 3 minutes video 8-10 are 1 minute. 
#actor 2 male: video 11-14 are 3 minutes 
#actor 3 female: video 15-18
#actor 4 male: video 19 is 3-minute
#actor 5 female: video 20 is 3-minute
# train_num_list = [1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
#train_num_list = [1, 2, 9, 5, 6, 7, 8, 10, 21, 22, 23, 24, 25, 26]
train_num_list = [1, 2, 4, 5, 6, 7, 9, 10]
val_num_list = [3]
combine_list = train_num_list + val_num_list
print(combine_list)

[1, 2, 4, 5, 6, 7, 9, 10, 3]


In [7]:
label_array_list = {}
txt_to_label = {'talk':0, 'eat':1, 'read':2, 'drink':3, 'computer':4, 'write':5, 'other': 6}
dirpath = "D:\\sensor_project\\2020_11_24\\video_sample\\sensor_data_v7\\"
manual_lable_path = dirpath + "manual_lable.csv"
manual_lable__csv = pd.read_csv(manual_lable_path)
n_lable_permin = 10000
for j in combine_list:
    #print(j)
    label_array = []
    label_permin = manual_lable__csv.iloc[:,j-1]
    for i in range(len(label_permin)):
        label = label_permin[i]
        #print(label)
        for k in range(n_lable_permin):
            label_array.append(label)
    label_array = np.asarray(label_array)
    label_array_list[j] = label_array

In [37]:
label_permin = manual_lable__csv.iloc[:,1]

In [10]:
label_array_list

{1: array([1, 1, 1, ..., 0, 0, 0], dtype=int64),
 2: array([0, 0, 0, ..., 3, 3, 3], dtype=int64),
 4: array([4, 4, 4, ..., 1, 1, 1], dtype=int64),
 5: array([1, 1, 1, ..., 4, 4, 4], dtype=int64),
 6: array([5, 5, 5, ..., 2, 2, 2], dtype=int64),
 7: array([2, 2, 2, ..., 1, 1, 1], dtype=int64),
 9: array([4, 4, 4, ..., 2, 2, 2], dtype=int64),
 10: array([2, 2, 2, ..., 2, 2, 2], dtype=int64),
 3: array([3, 3, 3, ..., 2, 2, 2], dtype=int64)}

In [18]:
"""
label_array_list = {}
# train_num = 21
txt_to_label = {'talk':0, 'eat':1, 'read':2, 'drink':3, 'computer':4, 'write':5, 'other': 6}
dirpath = "D:\\sensor_project\\2020_11_24\\video_sample\\sensor_data_v7\\"
#dirpath = "C:\\Users\\zhjsc\\Desktop\\zongyuan\\sensor\\repos\\time_series_network\\sensor_data_v7\\"

# dirpath = '/media/rzeng/Linzh/donghao/sensorv5/video_sample/sensor_data_v7/'
for j in combine_list:
    frontpath = dirpath + "Video" + str(j) + os.sep + "Video" + str(j) + os.sep + "front_angle.MOV_out.csv"
    front_csv = pd.read_csv(frontpath)
    backpath = dirpath + "Video" + str(j) + os.sep + "Video" + str(j) + os.sep + "back_angle.MOV_out.csv"
    back_csv = pd.read_csv(backpath)
    len_slowfast = min(front_csv.shape[0], back_csv.shape[0])
#     print(front_csv.shape, back_csv.shape)
    label_array = []
    for i in range(len_slowfast):
        front_row = front_csv.iloc[i]
        back_row = back_csv.iloc[i]
        label = compute_label(input_front=front_row, input_back=back_row, convert_dict=txt_to_label)
        label_array.append(label)
    label_array = np.asarray(label_array)
    label_array_list[j] = label_array
"""

In [11]:
def sensor_to_slowfast(sensor_index, sensor_data, label_data):
#     print(sensor_data.shape, label_data.shape)
    slowfast_index = int(sensor_index  / sensor_data.shape[0] * label_data.shape[0])
    return slowfast_index
window_sz = 128
sample_sz = 128
all_label_list = {}
head_sensor_data_list = {}
# train_num = 8
for j in combine_list:
    print('j', j)
    head_sensor_name = dirpath + os.sep + "Video" + str(j) + os.sep + "Video" + str(j) + os.sep + "Head.txt"
    head_sensor = load_sensor_data_without_h(fname=head_sensor_name)
    head_sensor_data = sample_sensor_data(head_sensor)
    sensor_length = head_sensor.shape[0]
    all_label = np.zeros((head_sensor_data.shape[0], 1)) 
    cnt = 0
    for k in range(0, sensor_length-window_sz-sample_sz, sample_sz):
        start = sensor_to_slowfast(sensor_index=k, sensor_data = head_sensor, label_data=label_array_list[j])
        end = sensor_to_slowfast(sensor_index=k+window_sz, sensor_data = head_sensor, label_data=label_array_list[j])
        label_data=label_array_list[j]
        cur_label_array = label_data[start:end]
        most_frequent_element = most_frequent(cur_label_array)
        all_label[cnt] = most_frequent_element
        cnt = cnt + 1
    all_label_list[j] = all_label
    head_sensor_data_list[j] = head_sensor_data

j 1
j 2
j 4
j 5
j 6
j 7
j 9
j 10
j 3


In [12]:
all_label

array([[3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],

In [13]:
all_label_list

{1: array([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
   

In [14]:
head_sensor_data_combine = head_sensor_data_list[train_num_list[0]]
all_label_combine = all_label_list[train_num_list[0]]
for train_num in train_num_list:
    head_sensor_data = head_sensor_data_list[train_num]
    head_sensor_data_combine = np.concatenate((head_sensor_data_combine,head_sensor_data), axis=0)
    all_label = all_label_list[train_num]
    all_label_combine = np.concatenate((all_label_combine, all_label), axis=0)
print(head_sensor_data_combine.shape)
print(all_label_combine.shape)

(4069, 13, 128)
(4069, 1)


In [16]:
all_label_combine.shape

(4069, 1)

In [17]:
cnt = 0
all_label_combine_filtered = []
remove_indices = []
for i in range(len(all_label_combine)):
    if all_label_combine[i] > 5.5 and all_label_combine[i] < 6.5:
        remove_indices.append(i)
all_label_combine_filtered = np.delete(all_label_combine, remove_indices, axis=0)
# print(all_label_combine_filtered.shape)
head_sensor_data_combine_filtered = np.delete(head_sensor_data_combine, remove_indices, axis=0)
print(head_sensor_data_combine_filtered.shape)

(4069, 13, 128)


In [20]:
all_label_list[val_num_list[0]].shape

(459, 1)

In [21]:
val_head_sensor_data = head_sensor_data_list[val_num_list[0]]
val_label = all_label_list[val_num_list[0]]
for val_num in val_num_list:
    head_sensor_data = head_sensor_data_list[val_num]
    val_head_sensor_data = np.concatenate((val_head_sensor_data, head_sensor_data), axis=0)
    all_label = all_label_list[val_num]
    val_label = np.concatenate((val_label, all_label), axis=0)
remove_indices = []
for i in range(len(val_label)):
    if val_label[i] > 5.5 and val_label[i] < 6.5:
        remove_indices.append(i)
val_label_filtered = np.delete(val_label, remove_indices, axis=0)
# print(val_label_filtered.shape)
val_head_sensor_data_filtered = np.delete(val_head_sensor_data, remove_indices, axis=0)
print('validation dataset shape', val_head_sensor_data_filtered.shape)

validation dataset shape (918, 13, 128)


In [24]:
val_label.shape

(918, 1)

In [25]:
bs = 10
sz = all_label.shape[0]
trn_ds = TensorDataset(torch.tensor(head_sensor_data_combine_filtered).float(), torch.tensor(all_label_combine_filtered).long())
trn_dl = DataLoader(trn_ds, batch_size=bs, shuffle=True, num_workers=0)

In [26]:
val_label = np.asarray(val_label)
val_ds = TensorDataset(torch.tensor(val_head_sensor_data_filtered).float(), torch.tensor(val_label_filtered).long())
val_dl = DataLoader(val_ds, batch_size=bs, shuffle=True, num_workers=0)

In [27]:
# print(np.unique(val_label_filtered))
# print(np.unique(all_label_combine_filtered))
all_label_list = []
val_label_list = []
for cur_label in all_label_combine_filtered:
    all_label_list.append(cur_label[0])
print('training set', Counter(all_label_list))
for cur_val_label in val_label_filtered:
    val_label_list.append(cur_val_label[0])
print('validating set', Counter(val_label_list))
print(txt_to_label)

training set Counter({5.0: 680, 0.0: 680, 4.0: 678, 1.0: 677, 2.0: 677, 3.0: 677})
validating set Counter({3.0: 154, 4.0: 154, 5.0: 154, 0.0: 154, 1.0: 152, 2.0: 150})
{'talk': 0, 'eat': 1, 'read': 2, 'drink': 3, 'computer': 4, 'write': 5, 'other': 6}


In [28]:
lr = 0.0001
n_epochs = 200
iterations_per_epoch = len(trn_ds)
num_classes = 6
best_acc = 0
patience, trials = 500, 0
base = 1
step = 2
loss_history = []
acc_history = []
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = Classifier_dh(head_sensor_data.shape[1], head_sensor_data.shape[2], num_classes).to(device)
criterion = nn.CrossEntropyLoss(reduction='sum')
opt = optim.Adam(model.parameters(), lr=lr)
print('Start model training')

for epoch in range(1, n_epochs + 1):
    model.train()
    epoch_loss = 0
    for i, batch in enumerate(trn_dl):
        x_raw, y_batch = [t.to(device) for t in batch]
        opt.zero_grad()
        out = model(x_raw)
        y_batch = tf.squeeze(y_batch)
#         print(out.size(), y_batch.size())
#         print('preds', preds, 'y_batch', y_batch)
        loss = criterion(out, y_batch)
        epoch_loss += loss.item()
        loss.backward()
        opt.step()

    loss_history.append(epoch_loss)
    model.eval()
    correct, total = 0, 0
    for batch in val_dl:
        x_raw, y_batch = [t.to(device) for t in batch]
        y_batch = tf.squeeze(y_batch)
        out = model(x_raw)
        preds = F.log_softmax(out, dim=1).argmax(dim=1)
#         print('preds', preds, 'y_batch', y_batch)
        if preds.size()[0] > 1:
            total += y_batch.size(0)
            correct += (preds == y_batch).sum().item()

    acc = correct / total
    acc_history.append(acc)
    if epoch % base == 0:
        print(f'Epoch: {epoch:3d}. Loss: {epoch_loss:.4f}. Acc.: {acc:2.2%}')
        base *= step

    if acc > best_acc:
        trials = 0
        best_acc = acc
        torch.save(model.state_dict(), 'best.pth')
        print(f'Epoch {epoch} best model saved with accuracy: {best_acc:2.2%}')
    else:
        trials += 1
        if trials >= patience:
            print(f'Early stopping on epoch {epoch}')
            break

Start model training
Epoch:   1. Loss: 7230.4133. Acc.: 37.25%
Epoch 1 best model saved with accuracy: 37.25%
Epoch:   2. Loss: 5605.4628. Acc.: 57.30%
Epoch 2 best model saved with accuracy: 57.30%
Epoch 3 best model saved with accuracy: 58.61%
Epoch:   4. Loss: 4601.2665. Acc.: 61.44%
Epoch 4 best model saved with accuracy: 61.44%
Epoch 7 best model saved with accuracy: 61.87%
Epoch:   8. Loss: 4032.6369. Acc.: 64.27%
Epoch 8 best model saved with accuracy: 64.27%
Epoch 10 best model saved with accuracy: 64.49%
Epoch 11 best model saved with accuracy: 69.06%
Epoch 12 best model saved with accuracy: 74.07%
Epoch 13 best model saved with accuracy: 74.73%
Epoch 14 best model saved with accuracy: 75.60%
Epoch 15 best model saved with accuracy: 77.12%
Epoch:  16. Loss: 2820.6029. Acc.: 78.43%
Epoch 16 best model saved with accuracy: 78.43%
Epoch 18 best model saved with accuracy: 80.17%
Epoch 19 best model saved with accuracy: 80.39%
Epoch 22 best model saved with accuracy: 81.70%
Epoch 2

In [29]:
model.load_state_dict(torch.load('best.pth'))
model.eval()

Classifier_dh(
  (conv1): SepConv1d(
    (layers): Sequential(
      (0): _SepConv1d(
        (depthwise): Conv1d(13, 13, kernel_size=(8,), stride=(2,), padding=(3,), groups=13)
        (pointwise): Conv1d(13, 32, kernel_size=(1,), stride=(1,))
      )
      (1): ReLU(inplace=True)
      (2): Dropout(p=0.05, inplace=False)
    )
  )
  (conv2): SepConv1d(
    (layers): Sequential(
      (0): _SepConv1d(
        (depthwise): Conv1d(32, 32, kernel_size=(8,), stride=(4,), padding=(2,), groups=32)
        (pointwise): Conv1d(32, 64, kernel_size=(1,), stride=(1,))
      )
      (1): ReLU(inplace=True)
      (2): Dropout(p=0.05, inplace=False)
    )
  )
  (conv3): SepConv1d(
    (layers): Sequential(
      (0): _SepConv1d(
        (depthwise): Conv1d(64, 64, kernel_size=(8,), stride=(4,), padding=(2,), groups=64)
        (pointwise): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
      )
      (1): ReLU(inplace=True)
      (2): Dropout(p=0.05, inplace=False)
    )
  )
  (conv4): SepConv1d(
  

In [30]:
# i = 1
preds_list = []
labels_list = []
for batch in val_dl:
    x_raw, y_batch = [t.to(device) for t in batch]
    y_batch = tf.squeeze(y_batch)
    out = model(x_raw)
    preds = F.log_softmax(out, dim=1).argmax(dim=1)
    preds_list = np.concatenate((preds_list, preds.cpu().numpy()), axis=0)
    labels_list = np.concatenate((labels_list, y_batch.cpu().numpy()), axis=0)
from sklearn.metrics import confusion_matrix
confusion_matrix(y_true=labels_list, y_pred=preds_list)

array([[146,   4,   2,   0,   0,   2],
       [  6, 134,   2,   6,   2,   2],
       [  0,   0, 142,   2,   0,   6],
       [  6,   4,   0, 142,   2,   0],
       [  0,   0,   8,   0, 140,   6],
       [  2,   0,   6,   0,   2, 144]], dtype=int64)